**NLP & Deep Learning Assignment 3**

This is a very interesting task of language modelling and I shall guide you at every single step of this assginment. 
Uptil the model architecture step, the code and logic written for data pre-processing and tokenization is similar to the NMT-Assigment. Feel free to skip to the model architecture step, where majority of the work is being done. 

**NOTE**: 

None of the code is plagiarized from any of the open source materials. Few functions are conventional statements while using keras framework and have been directly refered from its documentation.


In [0]:
import numpy as np
import pandas as pd
import re
import string
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM,Bidirectional
from keras.layers import Embedding

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Step 1: Reading Data**

---
The textual data provided that can be downloaded from here: ​https://drive.google.com/open?id=1RviYV0UP6OhPzBeLePetR05D4us3W0eU


We'll also be needing Word2Vec Glove Embeddings which can be easily downloaded from here: http://nlp.stanford.edu/data/glove.6B.zip




In [0]:
with open('/content/drive/My Drive/corpus.txt', 'r') as f:
    text = f.read()
    print(text)

One winter's evening, towards the close of the year 1800, or within a year
or two of that time, a young medical practitioner, recently established in
business, was seated by a cheerful fire in his little parlour, listening to the
wind which was beating the rain in pattering drops against the window, or
rumbling dismally in the chimney. The night was wet and cold; he had
been walking through mud and water the whole day, and was now
comfortably reposing in his dressing-gown and slippers, more than half
asleep and less than half awake, revolving a thousand matters in his
wandering imagination. First, he thought how hard the wind was blowing,
and how the cold, sharp rain would be at that moment beating in his face,
if he were not comfortably housed at home. Then, his mind reverted to
his annual Christmas visit to his native place and dearest friends; he
thought how glad they would all be to see him, and how happy it would
make Rose if he could only tell her that he had found a patient at l

**Step 2: Data Pre-Processing**

---
We'll be pre-processing to do the following tasks:
*   Removing numbers and digits from both input and output data
*   Removing Whitespaces in the sentence (leading and trailing)
*   Converting the entire sentence to lowercase
*   Remove all kinds of punctiontion items like ',{&%$' from the sentence. 

In [0]:
def clean_english_data(sentence):
  exclude = set(string.punctuation)
  remove_digits = str.maketrans('', '', string.digits)
  sentence = sentence.lower()
  sentence = ''.join(ch for ch in sentence if ch not in exclude)
  sentence = sentence.translate(remove_digits)
  sentence = sentence.strip()
  sentence = sentence.replace('\n',' ')
  sentence = re.sub(" +", " ", sentence)
  return sentence

In [0]:
text = text.split('. ')

In [0]:
text = [clean_english_data(x) for x in text]


In [0]:
text = ' '.join(text)
print(text)

one winters evening towards the close of the year or within a year or two of that time a young medical practitioner recently established in business was seated by a cheerful fire in his little parlour listening to the wind which was beating the rain in pattering drops against the window or rumbling dismally in the chimney the night was wet and cold he had been walking through mud and water the whole day and was now comfortably reposing in his dressinggown and slippers more than half asleep and less than half awake revolving a thousand matters in his wandering imagination first he thought how hard the wind was blowing and how the cold sharp rain would be at that moment beating in his face if he were not comfortably housed at home then his mind reverted to his annual christmas visit to his native place and dearest friends he thought how glad they would all be to see him and how happy it would make rose if he could only tell her that he had found a patient at last and hoped to have more a

**Step 3: Tokenization and Padding**


---
We simply tokenize the dataset and pad the sequences. For this assignment, I am using the 2 word sequence and its prediction


In [0]:
data = text
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [0]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 1346


In [0]:
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 4407


In [0]:
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 3


In [0]:
# split into input and output elements
sequences = np.array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

Creatng embedding weights using Word2Vec

In [0]:
word2Vec = {}
with open('/content/drive/My Drive/glove.6B.100d.txt') as f:
  for line in f:
    line = line.split(' ')
    word = line[0]
    word2Vec[word] = line[1:]

In [0]:
input_embedding_matrix = np.zeros((vocab_size,100))
for word, k in tokenizer.word_index.items():
  if k < vocab_size:
    embedding_vector = word2Vec.get(word)
    if embedding_vector is not None:
      input_embedding_matrix[k] = embedding_vector

**Step 4: Model Architecture**

---
The main challenge in the architecture this time was to use the right hyperparameters. Achieving 75-78% accuracy using normal RNN and LSTMs was not a big issue but to improve that score from 78% to 83% was a great hurdle. For the best results, we have used Bidirectional LSTM with 256 units. We have 100 dimmensions and relu for learning non-linear dependencies.
The dense layer has Softamax activation function which is used commanly in language modelling tasks.

I have also tried several neural nets with multiple LSTM layers and Bi-LSTM layer but they do not tend to improve the accuracy of the model and only increases the computational power.


In [0]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length-1, weights=[input_embedding_matrix]))
model.add(Bidirectional(LSTM(units=256,activation = 'relu')))

The key difference and improvement is to use 'Adagrad' optimizer over the most commonly used adam optimizer. Adagrad improves the overall accuracy of our model as it works best with a slow learning rate. More about the adagrad can be found here: https://medium.com/konvergen/an-introduction-to-adagrad-f130ae871827

In [0]:
model.add(Dropout(0.2))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='Adagrad', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 100)            134600    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               731136    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1346)              690498    
Total params: 1,556,234
Trainable params: 1,556,234
Non-trainable params: 0
_________________________________________________________________
None


Finally, we train our model. We achieve the highest accuracy of **83.25%** (epoch 340) which is being saved as best-case result and an average of 82.5% for the model. While traning earlier, I have also achieved as high as  **83.55%** for 500 epochs but that won't make huge difference in our prediction.  

In [0]:
model.fit(X, y, epochs=350, verbose=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/350
 - 5s - loss: 6.1689 - accuracy: 0.0860
Epoch 2/350
 - 4s - loss: 5.4235 - accuracy: 0.1087
Epoch 3/350
 - 4s - loss: 4.9326 - accuracy: 0.1262
Epoch 4/350
 - 4s - loss: 4.4840 - accuracy: 0.1477
Epoch 5/350
 - 4s - loss: 4.0768 - accuracy: 0.1618
Epoch 6/350
 - 4s - loss: 3.6781 - accuracy: 0.1895
Epoch 7/350
 - 4s - loss: 3.2999 - accuracy: 0.2280
Epoch 8/350
 - 4s - loss: 2.9187 - accuracy: 0.2889
Epoch 9/350
 - 4s - loss: 2.5784 - accuracy: 0.3703
Epoch 10/350
 - 4s - loss: 2.3060 - accuracy: 0.4252
Epoch 11/350
 - 4s - loss: 2.0617 - accuracy: 0.4885
Epoch 12/350
 - 4s - loss: 1.8712 - accuracy: 0.5255
Epoch 13/350
 - 4s - loss: 1.7138 - accuracy: 0.5541
Epoch 14/350
 - 4s - loss: 1.5985 - accuracy: 0.5875
Epoch 15/350
 - 4s - loss: 1.4940 - accuracy: 0.6118
Epoch 16/350
 - 4s - loss: 1.4011 - accuracy: 0.6276
Epoch 17/350
 - 4s - loss: 1.3153 - accuracy: 0.6408
Epoch 18/350
 - 4s - loss: 1.2593 - accuracy: 0.6546
Epoch 19/350
 - 4s - loss: 1.1922 - accuracy: 0.6764
Ep

In [0]:
model.save('/content/drive/My Drive/Checkpoints/final_model.h5')

**Step 5: Predction**

---
We generate and predict the sequence of words given a prior text sequence. On testing, we find that our model seems to perform pretty well for short and medium length sentences.  


In [0]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [0]:
print("Actual Text:\nOne winter's evening, towards the close of the year 1800, or within a year or two of that time")
print("Predicted Text:\n",generate_seq(model, tokenizer, max_length-1, 'towards the', 1))
print("Predicted Text:\n",generate_seq(model, tokenizer, max_length-1, 'towards the close', 3),"\n")
print("Actual Text:\n A companion, equally guilty with himself, had been acquitted for want of evidence")
print("Predicted Text:\n",generate_seq(model, tokenizer, max_length-1, 'a companion equally', 1))
print("Predicted Text:\n",generate_seq(model, tokenizer, max_length-1, 'a companion equally', 5),"\n")
print("Actual Text:\n With a motion so sudden, that the woman hardly knew that he had slipped from beside her")
print("Predicted Text:\n",generate_seq(model, tokenizer, max_length-1, 'a motion', 3),"\n")
print("Actual Text:\n This man died no natural or easy death,' said the surgeon.")
print("Predicted Text:\n",generate_seq(model, tokenizer, max_length-1, 'this man', 5),"\n")
print("Actual Text:\n The hysteric sobs which issued from beneath the veil, and the convulsive attitude of grief of the whole figure, however, at once exposed the absurdity of the suspicion; and he hastily followed.")
print("Predicted Text:\n",generate_seq(model, tokenizer, max_length-1, 'The hysteric sobs which issued from beneath the veil', 15))

Actual Text:
One winter's evening, towards the close of the year 1800, or within a year or two of that time
Predicted Text:
 towards the door
Predicted Text:
 towards the close of the metropolis 

Actual Text:
 A companion, equally guilty with himself, had been acquitted for want of evidence
Predicted Text:
 a companion equally guilty
Predicted Text:
 a companion equally guilty with himself had been 

Actual Text:
 With a motion so sudden, that the woman hardly knew that he had slipped from beside her
Predicted Text:
 a motion so sudden that 

Actual Text:
 This man died no natural or easy death,' said the surgeon.
Predicted Text:
 this man died no natural or easy 

Actual Text:
 The hysteric sobs which issued from beneath the veil, and the convulsive attitude of grief of the whole figure, however, at once exposed the absurdity of the suspicion; and he hastily followed.
Predicted Text:
 The hysteric sobs which issued from beneath the veil put it down without a sighwith cheerfulnesswith

As you can see in the first case, our model is not failing completely as towards the 'door' is also one of the sequences in the dataset. But when you increase the prediction length, it fails completely as metropolis is no where near the context of the sentence. In the 5th case, with a very long sequence it also fails completely. 

In all the other test cases, model seems to perform extremely well in predicting the right sentences. For the encoding sequence(2), it predicts just the right word.

Kindly let me know if there are any improvements needed in the code or the model.

**THANK YOU**


